In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import RandomizedSearchCV
from sklearn import metrics
import warnings
warnings.filterwarnings('ignore')

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Leitura e pré-processamento dos dados

In [ ]:
df = pd.read_csv('/kaggle/input/cardiovascular-disease-dataset/cardio_train.csv', sep=';')

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
# Convertendo peso e altura em IMC
df['IMC'] = df['weight']/((df['height']/100)**2)
df['IMC'] = df['IMC'].round(1)
# Convertendo a idade de datas para anos
df['age_year'] = df['age']/365
df['age_year'] = df['age_year'].astype(int)

In [ ]:
# Excluindo as colulas ano, altura, peso e id
df.drop(['age', 'height', 'weight', 'id'], axis=1, inplace=True)

In [ ]:
df.describe()

Com a descrição acima, pode-se verificar que as pressões alta e baixa (ap_hi e ap_lo), e o IMC estão com valores fora do normal (outliers)

In [ ]:
# Reformulando o dataframe para o boxplot
df_melt = pd.melt(df,id_vars='cardio', value_vars=['age_year', 'ap_hi', 'ap_lo', 'IMC'],  var_name='colunas', value_name='valores', col_level=None)

In [ ]:
# Verificando os outliers com boxplot
plt.figure(figsize=(12,8))
sns.boxplot(x='colunas', y='valores', hue='cardio', data=df_melt)

### Separando os dados com base na coluna cardio

In [ ]:
# Criando um dataframe para cardio igual a 0
df_c0 = df[df['cardio'] == 0]

In [ ]:
# Verificando o máximo e o mínimo dos quartis para identificar os outliers
lista = ['ap_lo', 'ap_hi', 'IMC']
df_quartil = pd.DataFrame(index=['Q minimo', 'Q maximo'])

for e in lista:
    Q1 = df_c0[e].quantile(0.25)
    Q3 = df_c0[e].quantile(0.75)
    IQR = Q3 - Q1
    
    minimo = Q1 - (1.5 * IQR)
    maximo = Q3+ (1.5 * IQR)
    
    df_quartil[e]= [minimo, maximo]
    
df_quartil

In [ ]:
# Excluindo as linhas que possuem os outliers
outlier_c0 = df_c0[(df_c0['ap_lo']!=80)|(df_c0['ap_hi']>121.5)|(df_c0['ap_hi']<117.5)|(df_c0['IMC']>36.65)|(df_c0['IMC']<15.45)].index
df_c0.drop(outlier_c0, inplace=True)

In [ ]:
df_c0.describe()

In [ ]:
# Criando um dataframe para cardio igual a 1
df_c1 = df[df['cardio'] == 1]

In [ ]:
# Verificando o máximo e o mínimo dos quartis para identificar os outliers
lista = ['ap_lo', 'ap_hi', 'IMC']
df_quartil = pd.DataFrame(index=['Q minimo', 'Q maximo'])

for e in lista:
    Q1 = df_c1[e].quantile(0.25)
    Q3 = df_c1[e].quantile(0.75)
    IQR = Q3 - Q1
    
    minimo = Q1 - (1.5 * IQR)
    maximo = Q3+ (1.5 * IQR)
    
    df_quartil[e]= [minimo, maximo]
    
df_quartil

In [ ]:
# Excluindo as linhas que possuem os outliers
outlier_c1 = df_c1[(df_c1['ap_lo']>105)|(df_c1['ap_lo']<65)|(df_c1['ap_hi']>170)|(df_c1['ap_hi']<90)|(df_c1['IMC']>39.65)|(df_c1['IMC']<14.45)].index
df_c1.drop(outlier_c1, inplace=True)

In [ ]:
df_c1.describe()

É possível verificar que as colunas, que antes possuiam outliers, agora estão com dados aceitáveis.

In [ ]:
# Concatenando os dados em um único dataframe
df = pd.concat([df_c0, df_c1])

In [ ]:
df_melt_new = pd.melt(df,id_vars='cardio', value_vars=['age_year', 'ap_hi', 'ap_lo', 'IMC'],  var_name='colunas', value_name='valores', col_level=None)

In [ ]:
# Verificando se os outliers foram excluídos utilizando boxplot
plt.figure(figsize=(12,8))
sns.boxplot(x='colunas', y='valores', hue='cardio', data=df_melt_new)

Com o boxplot fica mais fácil a visualização dos novos dados sem os outliers

In [ ]:
# Verificando quantos elementos tem em cada coluna, se baseando na coluna Cardio
sns.barplot(x='colunas', y='valores', hue='cardio',data=df_melt_new)

# Criação do modelo de Machine Learning

In [ ]:
# Separando os dados entre features e target
x = df.drop('cardio', axis=1)
y = df['cardio']

In [ ]:
knn = KNeighborsClassifier()
rf = RandomForestClassifier()
std = StandardScaler()
minmax = MinMaxScaler()

### Criação dos pipelines

In [ ]:

# Knn com standardScaler
pip_knn_std = Pipeline([
    ('scaler', std),
    ('clf', knn)
])

# Knn com MinMaxScaler
pip_knn_minmax = Pipeline([
    ('scaler', minmax),
    ('clf', knn)
])

# Random Forest com StandardScaler
pip_rf_std = Pipeline([
    ('scaler', std),
    ('clf', rf)
])

# Random Forest com MinMaxScaler
pip_rf_minmax = Pipeline([
    ('scaler', minmax),
    ('clf', rf)
])

In [ ]:
# Criação da lista de parâmetros
lista = list(range(10, 151))
# Aplicando as listas aos parâmetros
param_knn = {'clf__n_neighbors':lista}
param_rf = {'clf__n_estimators': lista}

In [ ]:
# Criação dos modelos
rs_knn_std = RandomizedSearchCV(pip_knn_std, param_knn, cv=5, verbose=1, random_state=42)
rs_knn_minmax = RandomizedSearchCV(pip_knn_minmax, param_knn, cv=5, verbose=1, random_state=42)
rs_rf_std = RandomizedSearchCV(pip_rf_std, param_rf, cv=5, verbose=1, random_state=42)
rs_rf_minmax = RandomizedSearchCV(pip_rf_minmax, param_rf, cv=5, verbose=1, random_state=42)

## Treinando os modelos e aplicando as métricas de avaliação

### Knn com standardScaler

In [ ]:
# Treinamento 
rs_knn_std.fit(x, y)

In [ ]:
# Matriz de confusão
print(pd.crosstab(y, rs_knn_std.predict(x), margins=True))

In [ ]:
# Relatório das métricas de classficação
print(metrics.classification_report(y, rs_knn_std.predict(x)))

### Knn com MinMaxScaler

In [ ]:
# Treinamento
rs_knn_minmax.fit(x, y)

In [ ]:
# Matriz de confusão
print(pd.crosstab(y, rs_knn_minmax.predict(x), margins=True))

In [ ]:
# Relatório das métricas de classficação
print(metrics.classification_report(y, rs_knn_minmax.predict(x)))

### Random Forest com StandardScaler

In [ ]:
# Treinamento
rs_rf_std.fit(x, y)

In [ ]:
# Matriz de confusão
print(pd.crosstab(y, rs_rf_std.predict(x), margins=True))

In [ ]:
# Relatório das métricas de classificação
print(metrics.classification_report(y, rs_rf_std.predict(x)))

### Random Forest com MinMaxScaler

In [ ]:
# Treinamento
rs_rf_minmax.fit(x, y)

In [ ]:
# Matriz de confusão
print(pd.crosstab(y, rs_rf_minmax.predict(x), margins=True))

In [ ]:
# Relatório das métricas de classificação
print(metrics.classification_report(y, rs_rf_minmax.predict(x)))

Os dois modelos feitos com Random forest possuem as maiores precisões, com 93%, enquando as com KNN atingiu 84%.  
Ambos levando em consideração a acurácia do F1-score no relatório das métricas de classificação